# Import libraries

In [1]:
!pip install -q numpy pandas luigi sqlalchemy dotenv psycopg2


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import csv
import subprocess
import luigi
from dotenv import load_dotenv
import os
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import types as sa_types
from sqlalchemy.orm import sessionmaker
from io import StringIO
import logging
import warnings
import time
from datetime import datetime
warnings.filterwarnings('ignore')

# Supporting functions

## Log to csv

In [2]:
def log_to_csv(log_msg: dict, filename: str):
    # Check if the file exists
    file_exists = os.path.isfile(filename)

    # Define the column headers
    headers = ["step", "status", "error_msg", "source", "table_name", "execution_time", "etl_date"]

    with open(filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)

        # Write the header only if the file doesn't exist
        if not file_exists:
            writer.writeheader()

        # Append the log message
        writer.writerow(log_msg)

## Read SQL File

In [3]:
def read_sql_file(file_path):
    """
    Reads an SQL file and returns its content as a string.

    Args:
        file_path (str): The path to the SQL file to be read.

    Returns:
        str or None: The content of the SQL file as a string if successful, 
                     or None if an error occurs.

    Note:
        Make sure to handle exceptions properly in the calling code.
        The function assumes that the SQL file is encoded as UTF-8.
    """
    try:
        with open(file_path, 'r') as file:
            sql_string = file.read()
        return sql_string
    except Exception as e:
        print(f"Error reading SQL file: {e}")
        return None

## Postgres connection

In [4]:
def db_connection():
    try:
        src_database = os.getenv("SRC_POSTGRES_DB")
        src_host = os.getenv("SRC_POSTGRES_HOST")
        src_user = os.getenv("SRC_POSTGRES_USER")
        src_password = os.getenv("SRC_POSTGRES_PASSWORD")
        src_port = os.getenv("SRC_POSTGRES_PORT")

        dwh_database = os.getenv("DWH_POSTGRES_DB")
        dwh_host = os.getenv("DWH_POSTGRES_HOST")
        dwh_user = os.getenv("DWH_POSTGRES_USER")
        dwh_password = os.getenv("DWH_POSTGRES_PASSWORD")
        dwh_port = os.getenv("DWH_POSTGRES_PORT")
        
        src_conn = f"postgresql://{src_user}:{src_password}@{src_host}:{src_port}/{src_database}"
        dwh_conn = f"postgresql://{dwh_user}:{dwh_password}@{dwh_host}:{dwh_port}/{dwh_database}"
        
        src_engine = create_engine(src_conn)
        dwh_engine = create_engine(dwh_conn)
        
        return src_engine, dwh_engine

    except Exception as e:
        print(f"Error: {e}")
        return None

## Move table from temp log to log directory

In [7]:
def move_temp_to_log_dir(
    logger, DIR_TEMP_LOG, DIR_LOG, etl_phase, log_time
):
    try:
        # Create destination directory if it doesn't exist
        subprocess.run(["mkdir", "-p", DIR_LOG], check=True)
        
        # Move all files from temp log to log directory
        result = subprocess.run(["mv", f"{DIR_TEMP_LOG}/{etl_phase}_log_{log_time}.log", DIR_LOG], 
                            shell=False, capture_output=True, text=True)
        
        if result.returncode == 0:
            logger.info(f"{etl_phase}_log_{log_time} moved to {DIR_LOG} successfully")
        else:
            logger.info(f"Error: {result.stderr}")
            
    except subprocess.CalledProcessError as e:
        logger.error(f"Command failed: {e}")
    except Exception as e:
        logger.error(f"Error: {e}")

# ETL functions

In [8]:
# Load environment variables from .env file
load_dotenv()

# Define DIR
DIR_ROOT_PROJECT = os.getenv("DIR_ROOT_PROJECT")
DIR_TEMP_LOG = os.getenv("DIR_TEMP_LOG")
DIR_TEMP_DATA = os.getenv("DIR_TEMP_DATA")
DIR_EXTRACT_QUERY = os.getenv("DIR_EXTRACT_QUERY")
DIR_LOAD_QUERY = os.getenv("DIR_LOAD_QUERY")
DIR_DBT_TRANSFORM = os.getenv("DIR_DBT_TRANSFORM")
DIR_LOG = os.getenv("DIR_LOG")

print(f"DIR_ROOT_PROJECT: {DIR_ROOT_PROJECT}")
print(f"DIR_TEMP_LOG: {DIR_TEMP_LOG}")
print(f"DIR_TEMP_DATA: {DIR_TEMP_DATA}")
print(f"DIR_EXTRACT_QUERY: {DIR_EXTRACT_QUERY}")
print(f"DIR_LOAD_QUERY: {DIR_LOAD_QUERY}")
print(f"DIR_LOAD_QUERY: {DIR_DBT_TRANSFORM}")
print(f"DIR_LOG: {DIR_LOG}")

DIR_ROOT_PROJECT: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store
DIR_TEMP_LOG: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store/pipeline/temp/logs
DIR_TEMP_DATA: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store/pipeline/temp/data
DIR_EXTRACT_QUERY: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store/pipeline/src_query/extract
DIR_LOAD_QUERY: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store/pipeline/src_query/load
DIR_LOAD_QUERY: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store/pipeline/src_query/dbt_transform
DIR_LOG: /home/biyanbahtiar/pacmann/data_storage/dbt_x_luigi/pacbook_store/logs


In [9]:
class GlobalParams(luigi.Config):
    tables_to_modify = [
        "country",
        "address_status",
        "author",
        "book_language",
        "publisher",
        "shipping_method",
        "order_status",
        "customer",
        "order_history",
        "order_line",
        "book_author",
        "customer_address",
        "address",
        "book",
        "cust_order",
    ]

    result_tables = [
        "dim_customer",
        "dim_address",
        "dim_book",
        "dim_author",
        "bridge_book_author",
        "dim_date",
        "dim_time",
        "fct_book_order",
        "fct_customer_behavior"
    ]

    CurrentTimestampParams = luigi.DateSecondParameter(default = datetime.now())

In [ ]:
class Extract(luigi.Task):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.get_current_timestamp = GlobalParams().CurrentTimestampParams
        self.table_to_extract = [
            "public."+table_name for table_name in GlobalParams().tables_to_modify
        ]

    def requires(self):
        pass

    def run(self):
        logger = logging.getLogger(f"extract_log_{self.get_current_timestamp}.log")

        # Check if handlers are already set to avoid duplicate logs on re-runs
        if not logger.handlers:
            handler = logging.FileHandler(f"{DIR_TEMP_LOG}/extract_log_{self.get_current_timestamp}.log")
            formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)
        
        try:
            # Define db connection engine
            src_engine, _ = db_connection()

            # Define the query using the SQL content
            extract_query = read_sql_file(
                file_path = f"{DIR_EXTRACT_QUERY}/all-tables.sql"
            )

            logger.info("==================================STARTING EXTRACT DATA=======================================")
            for table_name in self.table_to_extract:
                start_time = time.time()

                try:
                    start_time_extract = time.time()
                    # Read data into Dataframe
                    df = pd.read_sql_query(extract_query.format(table_name=table_name), src_engine)
                    
                    # Write dataframe to .csv
                    df.to_csv(f"{DIR_TEMP_DATA}/{table_name.replace('public.', '')}.csv", index=False)
                    logger.info(f"EXTRACT {table_name} - SUCCESS.")

                    end_time = time.time()  # Record end time
                    execution_time = end_time - start_time_extract  # Calculate execution time
                    
                    # Get summary
                    summary_data = {
                        "step":"extraction",
                        "status":"success",
                        "error_msg":"",
                        "source":"pacbook_transactional_db",
                        "table_name":table_name,
                        "execution_time":execution_time,
                        "etl_date":self.get_current_timestamp
                    }

                    # load log to csv file
                    log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")

                except Exception as e:
                    end_time = time.time()  # Record end time
                    execution_time = end_time - start_time  # Calculate execution time
                    
                    # Get summary
                    summary_data = {
                        "step":"extraction",
                        "status":"failed",
                        "error_msg":str(e),
                        "source":"pacbook_transactional_db",
                        "table_name":table_name,
                        "execution_time":execution_time,
                        "etl_date":self.get_current_timestamp
                    }

                    # load log to csv file
                    log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")
                    logger.error(f"Failed to extract {table_name} tables")
                    raise Exception(f"Failed to extract {table_name} tables: {e}")
                  
        except Exception as e:
            end_time = time.time()
            execution_time = end_time - start_time
             
            # Get summary
            summary_data = {
                "step":"extraction",
                "status":"failed",
                "error_msg":str(e),
                "source":"pacbook_transactional_db",
                "table_name":self.table_to_extract,
                "execution_time":execution_time,
                "etl_date":self.get_current_timestamp
            }
            
            # load log to csv file
            log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")
            
            # Write exception
            logger.error(f"Extract All Tables From Sources - FAILED")
            raise Exception(f"FAILED to execute EXTRACT TASK: {e}")
        
        logger.info("==================================ENDING EXTRACT DATA=======================================")
        
        move_temp_to_log_dir(logger, DIR_TEMP_LOG, DIR_LOG, etl_phase="extract", log_time=self.get_current_timestamp)

    def output(self):
        outputs = []
        for table_name in self.table_to_extract:
            outputs.append(luigi.LocalTarget(f"{DIR_TEMP_DATA}/{table_name.replace('public.', '')}.csv"))

        outputs.append(luigi.LocalTarget(f"{DIR_LOG}/etl_log.csv"))

        outputs.append(luigi.LocalTarget(f"{DIR_LOG}/extract_log_{self.get_current_timestamp}.log"))
        return outputs

SyntaxError: f-string: unmatched '(' (3699571493.py, line 42)

In [ ]:
class Load(luigi.Task):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.get_current_timestamp = GlobalParams().CurrentTimestampParams

    def requires(self):
        return Extract()
        
    def run(self):
        logger = logging.getLogger(f"load_log_{self.get_current_timestamp}.log")

        # Check if handlers are already set to avoid duplicate logs on re-runs
        if not logger.handlers:
            handler = logging.FileHandler(f"{DIR_TEMP_LOG}/load_log_{self.get_current_timestamp}.log")
            formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)

        # ------ 1. Define SQL files path ------
        truncate_sql_file = "truncate_tables.sql"
        
        table_configs = {
            table_name: {"sql_file":f"staging-{table_name}.sql"}
            for table_name in GlobalParams().tables_to_modify
        }
        
        # ------ 2. Read SQL file and store in variable ------
        truncate_queries_content = ""

        try:
            # Read the truncate SQL file
            truncate_queries_content = read_sql_file(
                file_path = f"{DIR_LOAD_QUERY}/{truncate_sql_file}"
            )
    
            logger.info("Read Truncate Query - SUCCESS")
    
        except Exception as e:
            logger.error("Read Truncate Query - FAILED")
            raise Exception(f"Failed to read Truncate SQL queries: {e}")

        # ------ 3. Read extracted CSV data ------
        input_paths = [i.path for i in self.input()]

        try:
            for table_name, config in table_configs.items():
                expected_file_path = f"{DIR_TEMP_DATA}/{table_name}.csv"
                if expected_file_path in input_paths:
                    config["df"] = pd.read_csv(expected_file_path)
            logger.info("Read extracted CSV - SUCCESS")
            
        except Exception as e:
            logger.error("Read extracted CSV - FAILED")
            raise Exception(f"Failed to read CSV: {e}")
            
        # ------ 4. Execute queries to create schema if not exists and truncate ------
        # Establish connections to DWH
        dwh_engine = None
        try:
            _, dwh_engine = db_connection()
            logger.info(f"Connect to DWH - SUCCESS")
            
        except Exception:
            logger.error(f"Connect to DWH - FAILED")
            raise Exception("Failed to connect to Data Warehouse")

        # Create staging schema and truncate all tables before load
        try:            
            # Create session
            Session = sessionmaker(bind = dwh_engine)
            session = Session()

            # Split the SQL queries if multiple queries are present
            truncate_query = truncate_queries_content.split("--")
    
            # Remove newline characters and leading/trailing whitespaces
            truncate_query = [query.strip() for query in truncate_query if query.strip()]
            
            # Execute create schema query and truncate query
            for query in truncate_query:
                query = sqlalchemy.text(query)
                session.execute(query)

            session.commit()
            session.close()

            logger.info(f"Create/Truncate staging Schema in DWH - SUCCESS")
        
        except Exception as e:
            logger.error(f"Create/Truncate staging Schema in DWH - FAILED : {e}")
            raise Exception("Failed to Create/Truncate public Schema in DWH")
        
        #----------------------------------------------------------------------------------------------------------------------------------------
        # 5. Load data into staging schema DWH
        # Record start time for loading tables
        start_time = time.time()  
        logger.info("==================================STARTING LOAD DATA=======================================")
        # Load to tables
        try:
            #----------------------------------------------------------------------------------------------------------------------------------------
            try:
                # Create session
                Session = sessionmaker(bind = dwh_engine)
                session = Session()
   
                for table_name, config in table_configs.items():
                    start_time_load = time.time()
                    config["df"].to_sql(
                        table_name,
                        con = dwh_engine,
                        if_exists = "append",
                        index = False,
                        schema = "staging"
                    )

                    # Record end time for loading tables
                    end_time = time.time()  
                    execution_time = end_time - start_time_load
                    
                    # Get summary
                    summary_data = {
                        "step":"load",
                        "status":"success",
                        "error_msg":"",
                        "source":"extracted csv from Extract()",
                        "table_name":table_name,
                        "execution_time":execution_time,
                        "etl_date":self.get_current_timestamp
                    }

                    logger.info(f"LOAD Table {table_name} To staging schema DWH - SUCCESS")
                    
                    # load log to csv file
                    log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")
                
                logger.info(f"LOAD All Tables To staging schema DWH - SUCCESS")
                
            except Exception as e:
                logger.error(f"LOAD All Tables To staging schema DWH - FAILED: {e}")
                raise Exception("Failed Load Tables To staging schema DWH")
            
        #----------------------------------------------------------------------------------------------------------------------------------------
        except Exception as e:
            end_time = time.time()  
            execution_time = end_time - start_time
            
            # Get summary
            summary_data = {
                "step":"load",
                "status":"failed",
                "error_msg":str(e),
                "source":"extracted csv in temp",
                "table_name":list(table_configs.keys()),
                "execution_time":execution_time,
                "etl_date":self.get_current_timestamp
            }
            
            # load log to csv file
            log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")
            
            logger.error("LOAD All Tables To DWH - FAILED")
            raise Exception("Failed Load Tables To DWH")   
        
        logger.info("==================================ENDING LOAD DATA=======================================")

    move_temp_to_log_dir(logger, DIR_TEMP_LOG, DIR_LOG, etl_phase="load", log_time=self.get_current_timestamp)
        
    #----------------------------------------------------------------------------------------------------------------------------------------
    def output(self):
        return [luigi.LocalTarget(f"{DIR_LOG}/load_log_{self.get_current_timestamp}.log"),
                luigi.LocalTarget(f"{DIR_LOG}/etl_log.csv")]

In [ ]:
class Transform(luigi.Task):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.get_current_timestamp = GlobalParams().CurrentTimestampParams
        self.result_tables = GlobalParams().result_tables
    
    def requires(self):
        return Load()
    
    def run(self):
        logger = logging.getLogger(f'transform_log_{self.get_current_timestamp}.log')

        # Check if handlers are already set to avoid duplicate logs on re-runs
        if not logger.handlers:
            handler = logging.FileHandler(f'{DIR_TEMP_LOG}/transform_log_{self.get_current_timestamp}.log')
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)

        #----------------------------------------------------------------------------------------------------------------------------------------
        # Record start time for transform tables
        start_time = time.time()
        logging.info("==================================STARTING TRANSFROM DATA=======================================")  
               
        # Transform to dimensions tables
        try:
            with open (file = f'{DIR_TEMP_LOG}/transform_log_{self.get_current_timestamp}.log', mode = 'a') as f :
                subprocess.run(
                    f"cd {DIR_DBT_TRANSFORM} && dbt debug && dbt deps && dbt seed && dbt run && dbt snapshot && dbt test",
                    stdout = f,
                    stderr = subprocess.PIPE,
                    text = True,
                    shell = True,
                    check = True
                )
        
            # Record end time for loading tables
            end_time = time.time()  
            execution_time = end_time - start_time
            logging.info(f"Transform to All Dimensions and Fact Tables - SUCCESS")

            # Get summary
            summary_data = {
                "step":"transform",
                "status":"success",
                "error_msg":"",
                "source":"staging schema pacbook_dwh",
                "table_name":self.result_tables,
                "execution_time":execution_time,
                "etl_date":self.get_current_timestamp
            }
            
            # load log to csv file
            log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")
            
        except Exception as e:
            logging.error(f"Transform to All Dimensions and Fact Tables - FAILED")
        
            # Get summary
            summary_data = {
                "step":"transform",
                "status":"failed",
                "error_msg":str(e),
                "source":"staging schema pacbook_dwh",
                "table_name":self.result_tables,
                "execution_time":execution_time,
                "etl_date":self.get_current_timestamp
            }
            
            # load log to csv file
            log_to_csv(summary_data, f"{DIR_LOG}/etl_log.csv")
            
            logging.error(f"Transform Tables - FAILED: {e}")
            raise Exception('Failed Transforming Tables')   
        
        logging.info("==================================ENDING TRANSFROM DATA=======================================")

        move_temp_to_log_dir(logger, DIR_TEMP_LOG, DIR_LOG, etl_phase="transform", log_time=self.get_current_timestamp)

    #----------------------------------------------------------------------------------------------------------------------------------------
    def output(self):
        return [luigi.LocalTarget(f"{DIR_LOG}/transform_log_{self.get_current_timestamp}.log"),
                luigi.LocalTarget(f"{DIR_LOG}/etl_log.csv")]

In [ ]:
if __name__ == '__main__':
    luigi.build([Extract(), Load(), Transform()], local_scheduler = True)

DEBUG: Checking if Extract() is complete
INFO: Informed scheduler that task   Extract__99914b932b   has status   DONE
DEBUG: Checking if Load() is complete
DEBUG: Checking if Extract() is complete
INFO: Informed scheduler that task   Load__99914b932b   has status   PENDING
INFO: Informed scheduler that task   Extract__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 121497] Worker Worker(salt=2845575323, workers=1, host=LAPTOP-JMN90Q2L, username=biyanbahtiar, pid=121497) running   Load()
INFO: [pid 121497] Worker Worker(salt=2845575323, workers=1, host=LAPTOP-JMN90Q2L, username=biyanbahtiar, pid=121497) done      Load()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   Load__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt